# El Gran Dalmuti - QLearning

In [1]:
import numpy as np

### Basic game mechanics

In [2]:
NUM_CARD_VALUES = 13
JOKER = 12  # Jokers at index 12


def has_already_won(hand):
    """
    hand - vector with 13 entries (number of 1, 2, ..., 12, Jokers)
    """

    if len(hand.shape) == 1:
        return np.all(hand == 0)
    else:
        return np.all(hand == 0, axis=1)
    
    
def get_cards_array(card_type, num_cards):
    """ Vector representation of the cards of one kind. """
    
    cards_array = np.zeros(NUM_CARD_VALUES, dtype=np.int8)
    cards_array[card_type] = num_cards
    return cards_array
    

def possible_next_moves(hand, board):
    """
    Returns possible next moves as a list of tuples (new hand, new board)
    """
    
    card_type_in_board = np.argmax(board)
    num_cards_in_board = board[card_type_in_board] + board[JOKER]
    
    # You can always pass if it is not the initial move
    possible_hands = np.reshape(hand, (1, NUM_CARD_VALUES))
    possible_boards = np.reshape(board, (1, NUM_CARD_VALUES))
    
    if not has_already_won(hand):
        for card_type_in_hand in range(NUM_CARD_VALUES - 1, -1, -1):
            # You can play clean
            if card_type_in_hand < card_type_in_board and \
              hand[card_type_in_hand] >= num_cards_in_board:
                new_board = get_cards_array(card_type_in_hand, num_cards_in_board)
                new_hand = hand - new_board
                possible_hands = np.vstack([possible_hands, new_hand])
                possible_boards = np.vstack([possible_boards, new_board])

            # Or you can play dirty (with Joker(s))
            if card_type_in_hand != JOKER and hand[JOKER] > 0 and \
              card_type_in_hand < card_type_in_board and hand[card_type_in_hand] > 0 and \
              hand[card_type_in_hand] + hand[JOKER] >= num_cards_in_board:
                # Use one joker
                if hand[card_type_in_hand] + 1 >= num_cards_in_board:
                    joker_vec = get_cards_array(JOKER, 1)
                    new_board = get_cards_array(card_type_in_hand, num_cards_in_board - 1) + joker_vec
                    new_hand = hand - new_board
                    possible_hands = np.vstack([possible_hands, new_hand])
                    possible_boards = np.vstack([possible_boards, new_board])

                # Use two jokers
                if hand[JOKER] == 2 and num_cards_in_board > 2:
                    joker_vec = get_cards_array(JOKER, 2)
                    new_board = get_cards_array(card_type_in_hand, num_cards_in_board - 2) + joker_vec
                    new_hand = hand - new_board
                    possible_hands = np.vstack([possible_hands, new_hand])
                    possible_boards = np.vstack([possible_boards, new_board])
            
    return possible_hands, possible_boards
        
    
# Some tests
assert has_already_won(np.zeros(NUM_CARD_VALUES))
assert not has_already_won(np.ones(NUM_CARD_VALUES))
assert np.all(has_already_won(np.zeros((2, NUM_CARD_VALUES))) == np.array([True, True]))
assert np.all(has_already_won(np.array([[0,0,0,0,1], [0,0,0,0,0], [1,0,0,0,0]])) == np.array([False, True, False]))

assert np.all(get_cards_array(1, 2) == np.array([0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
assert np.all(get_cards_array(4, 3) == np.array([0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0]))
assert not np.all(get_cards_array(4, 3) == np.array([0, 2, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0]))

# Tests for possible moves
h, b = possible_next_moves(np.array([0, 2, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2]),
                           np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0]))
assert np.all(h == np.array([[0., 2., 0., 0., 0., 0., 0., 2., 0., 2., 0., 0., 2.],
                             [0., 2., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 2.],
                             [0., 2., 0., 0., 0., 0., 0., 1., 0., 2., 0., 0., 1.],
                             [0., 0., 0., 0., 0., 0., 0., 2., 0., 2., 0., 0., 2.],
                             [0., 1., 0., 0., 0., 0., 0., 2., 0., 2., 0., 0., 1.]]))
assert np.all(b == np.array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0.],
                             [0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0.],
                             [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.],
                             [0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                             [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]))
h, b = possible_next_moves(np.array([1, 2, 3, 1, 0, 0, 0, 3, 0, 4, 0, 0, 2]),
                           np.array([0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1]))
assert np.all(h == np.array([[1., 2., 3., 1., 0., 0., 0., 3., 0., 4., 0., 0., 2.],
                             [1., 2., 3., 1., 0., 0., 0., 0., 0., 4., 0., 0., 2.],
                             [1., 2., 3., 1., 0., 0., 0., 1., 0., 4., 0., 0., 1.],
                             [1., 2., 3., 1., 0., 0., 0., 2., 0., 4., 0., 0., 0.],
                             [1., 2., 3., 0., 0., 0., 0., 3., 0., 4., 0., 0., 0.],
                             [1., 2., 0., 1., 0., 0., 0., 3., 0., 4., 0., 0., 2.],
                             [1., 2., 1., 1., 0., 0., 0., 3., 0., 4., 0., 0., 1.],
                             [1., 2., 2., 1., 0., 0., 0., 3., 0., 4., 0., 0., 0.],
                             [1., 0., 3., 1., 0., 0., 0., 3., 0., 4., 0., 0., 1.],
                             [1., 1., 3., 1., 0., 0., 0., 3., 0., 4., 0., 0., 0.],
                             [0., 2., 3., 1., 0., 0., 0., 3., 0., 4., 0., 0., 0.]]))
assert np.all(b == np.array([[0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 1.],
                             [0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0.],
                             [0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 1.],
                             [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 2.],
                             [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 2.],
                             [0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                             [0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
                             [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2.],
                             [0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
                             [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2.],
                             [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2.]]))

In [3]:
h, b = possible_next_moves(np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 2]),
                           np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]))
len(h)

31

### Generate Random Initial Game States

In [4]:
NUM_PLAYERS = 4
AVAILABLE_CARDS = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 2]
PLAYER = list(range(NUM_PLAYERS))


def random_initial_cards():
    """ Random initial state for the game. """
    
    deck = np.array([], dtype=np.int8)

    for card_type in range(NUM_CARD_VALUES):
        deck = np.append(deck, np.array([card_type for _ in range(AVAILABLE_CARDS[card_type])]))
    
    np.random.shuffle(deck)
    
    chunk = deck.shape[0] // NUM_PLAYERS
    remainder = deck.shape[0] % NUM_PLAYERS
    first_player_initialized = False
    
    for playerIndex in range(NUM_PLAYERS):
        beginOfChunk = playerIndex * chunk + min(playerIndex, remainder)
        endOfChunk = (playerIndex + 1) * chunk + min(playerIndex + 1, remainder)
        player = np.zeros(NUM_CARD_VALUES, dtype=np.int8)
        
        for card in deck[beginOfChunk:endOfChunk]:
            player += get_cards_array(card, 1)
            
        if first_player_initialized:
            player_initial_hands = np.vstack([player_initial_hands, player])
        else:
            first_player_initialized = True
            player_initial_hands = player
        
    return player_initial_hands
        
        
random_initial_cards()

array([[0, 1, 0, 2, 2, 1, 3, 0, 3, 4, 1, 3, 0],
       [0, 1, 1, 1, 2, 1, 1, 2, 2, 4, 3, 1, 1],
       [1, 0, 2, 1, 1, 2, 0, 2, 2, 1, 3, 4, 1],
       [0, 0, 0, 0, 0, 2, 3, 4, 2, 1, 4, 4, 0]], dtype=int8)

In [5]:
np.sum(random_initial_cards(), axis=1) == 20 * np.ones(4)

array([ True,  True,  True,  True])

In [6]:
np.all(np.sum(random_initial_cards(), axis=1) == 20 * np.ones(NUM_PLAYERS))

True

### Encode Cards Vector for Indexing and Mapping

In [7]:
card_bit_offset = [
      0,  2,  6, 10, 14, 18,  22,  25,  28,  31,  34,  36,  38,
     39, 41, 45, 49, 53, 57,  61,  64,  67,  70,  73,  75,  77,
     78, 80, 84, 88, 92, 96, 100, 103, 106, 109, 112, 114, 116,
    117  # sentinel
]

def encode_card_array(card_array):
    """
    Encodes a card vector into an int64.
    | Board | Hand | Already played |
     38   26 25  13 12             0
    """
    
    pass


def decode_card_repr(card_repr):
    """
    Decodes an int64 back to a card vector.
    """
    
    pass



### Neural Network

In [9]:
import os

# Use GPU for model training
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

import numpy as np
import pandas as pd

from keras import models
from keras import layers

In [10]:
# Create sequential model
network = models.Sequential()

# Use to convolutional layers to group same cards
# of a kind among the different input vectors
network.add(layers.convolutional.Conv2D(
    # Input rows: Already played, board, hand, action
    input_shape=(4, 13, 1),
    # Window size
    kernel_size=(3, 3),
    # Output filters
    filters=11,
    # Activation
    activation='relu',
    #data_format='channels_first'
))

network.add(layers.convolutional.Conv2D(
    # Window size
    kernel_size=(2, 2),
    # Output filters
    filters=13,
    # Activation
    activation='relu'
))

# Flatten convolutional layers
network.add(layers.Flatten())
network.add(layers.Dense(13, activation="relu"))

# Dropout to remove noise
network.add(layers.Dropout(0.5))

# Final dense layer for q-value
network.add(layers.Dense(1))

# Compile neural network
network.compile(loss='mse', optimizer='RMSprop', metrics=['mse'])

network.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 2, 11, 11)         110       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 10, 13)         585       
_________________________________________________________________
flatten_1 (Flatten)          (None, 130)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 13)                1703      
_________________________________________________________________
dropout_1 (Dropout)          (None, 13)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 14        
Total params: 2,412
Tr

In [81]:
ex_al_ply = np.array(list(map(lambda li: [li], np.array([0,0,2,0,4,0,0,0,0,0,0,0,0]))))
ex_brd = np.array(list(map(lambda li: [li], np.array([0,0,2,0,0,0,0,0,0,0,0,0,0]))))
ex_hand = np.array(list(map(lambda li: [li], np.array([0,0,0,0,0,0,0,0,0,0,0,0,0]))))
ex_action = np.array(list(map(lambda li: [li], np.array([0,2,0,0,0,0,0,0,0,0,0,0,0]))))
ex_rew = np.array(list(map(lambda li: [li], np.array([1.]))))

x = np.stack([[ex_al_ply, ex_brd, ex_hand, ex_action]], axis=0)
y = ex_rew

In [75]:
np.array([[1,2],[2,3]]) == np.array([[1,2],[2,3]])

array([[ True,  True],
       [ True,  True]])

In [69]:
network.fit(x, y, epochs=1)

Epoch 1/1
1/1 [==============================] - 0s 0us/step - loss: 0.0323 - mean_squared_error: 0.0323


In [74]:
network.predict(x)[0, 0]

0.852979